# Imports

In [1]:
import random
import pandas as pd
import numpy as np
from copy import deepcopy

# Local Data

In [2]:
df = pd.read_csv("data/players.csv")
df = df.drop(columns=['Unnamed: 0']) # drop the index column

# Problem Configuration

In [3]:
TEAM_SIZE = 7
NUM_TEAMS = 5
BUDGET_LIMIT = 750
TEAM_STRUCTURE = {"GK": 1, "DEF": 2, "MID": 2, "FWD": 2}
POPULATION_SIZE = 10

# Building Classes

In [4]:
class Player:
    def __init__(self, name, position, skill, salary):
        self.name = name
        self.position = position
        self.skill = skill
        self.salary = salary

    @classmethod
    def from_dict(cls, data):
        return cls(
            name=data['Name'],
            position=data['Position'],
            skill=data['Skill'],
            salary=data['Salary (€M)']
        )

    def to_dict(self):
        return {
            'Name': self.name,
            'Position': self.position,
            'Skill': self.skill,
            'Salary (€M)': self.salary
        }

    def __repr__(self):
        return f"{self.position}: {self.name} | Skill: {self.skill} | Salary: €{self.salary}M"

In [5]:
class Team:
    def __init__(self, players):
        self.players = players  # List of Player objects

    def is_valid(self, structure, budget):
        if len(self.players) != sum(structure.values()):
            return False

        pos_counts = {pos: 0 for pos in structure}
        total_salary = 0
        names_seen = set()

        for player in self.players:
            if player.name in names_seen:
                return False
            names_seen.add(player.name)
            pos_counts[player.position] += 1
            total_salary += player.salary

        return pos_counts == structure and total_salary <= budget

    def avg_skill(self):
        return sum(p.skill for p in self.players) / len(self.players)

    def total_salary(self):
        return sum(p.salary for p in self.players)

    def player_names(self):
        return [p.name for p in self.players]

    def __repr__(self):
        return "\n".join([f"  - {p}" for p in self.players])

In [6]:
class LeagueIndividual:
    def __init__(self, players_by_position, team_structure, budget_limit, league=None):
        
        self.players_by_position = players_by_position
        self.team_structure = team_structure
        self.budget_limit = budget_limit

        self.league = league if league is not None else self._generate_league()
        self.fitness = self.evaluate_fitness()

    def _generate_league(self):
        league = []
        all_players = deepcopy(self.players_by_position)
        used_names = set()

        for _ in range(NUM_TEAMS):
            team_players = []

            for pos, count in self.team_structure.items():
                candidates = [p for p in all_players[pos] if p.name not in used_names]
                if len(candidates) < count:
                    return None  # Not enough players available

                selected = random.sample(candidates, count)
                team_players.extend(selected)
                used_names.update(p.name for p in selected)

            team = Team(team_players)
            if not team.is_valid(self.team_structure, self.budget_limit):
                return None

            league.append(team)

            # Remove used players from pool
            for pos in self.players_by_position:
                all_players[pos] = [p for p in all_players[pos] if p.name not in used_names]

        return league

    def evaluate_fitness(self):
        if self.league is None:
            return float('inf')

        avg_skills = []
        used_names = set()

        for team in self.league:
            if not team.is_valid(self.team_structure, self.budget_limit):
                return float('inf')

            for p in team.players:
                if p.name in used_names:
                    return float('inf')  # Duplicate player across teams
                used_names.add(p.name)

            avg_skills.append(team.avg_skill())

        return np.std(avg_skills)

    def __lt__(self, other):
        return self.fitness < other.fitness

    def __repr__(self):
        return f"<LeagueIndividual fitness={self.fitness:.4f}>"

# Convert DF to player objects

In [7]:
players_by_position = {
    pos: [Player.from_dict(row) for _, row in df[df['Position'] == pos].iterrows()]
    for pos in TEAM_STRUCTURE
}

# Generate Population

In [8]:
# === GENERATE POPULATION ===
def generate_initial_population(size, players_by_position, team_structure, budget_limit):
    population = []
    attempts = 0
    max_attempts = 1000 # avoid infinite loop if unable to generate valid leagues

    while len(population) < size and attempts < max_attempts:
        indiv = LeagueIndividual(players_by_position, team_structure, budget_limit)
        if indiv.league is not None:
            population.append(indiv)
        attempts += 1

    return population

population = generate_initial_population(
    POPULATION_SIZE,
    players_by_position,
    TEAM_STRUCTURE,
    BUDGET_LIMIT
)

# Check if Classes are working

In [9]:
# === EXAMPLE USAGE ===
individual = LeagueIndividual(players_by_position, TEAM_STRUCTURE, BUDGET_LIMIT)

# Print result
print("\n=== LeagueIndividual ===")
for i, team in enumerate(individual.league):
    print(f"\n🏆 Team {i + 1}")
    print(team)
    print(f"Avg Skill: {team.avg_skill():.2f} | Total Salary: €{team.total_salary()}M")

print(f"\nFitness: {individual.fitness:.4f}")


=== LeagueIndividual ===

🏆 Team 1
  - GK: Chris Thompson | Skill: 80 | Salary: €80M
  - DEF: Logan Brooks | Skill: 86 | Salary: €95M
  - DEF: Owen Parker | Skill: 88 | Salary: €100M
  - MID: Connor Hayes | Skill: 89 | Salary: €105M
  - MID: Nathan Wright | Skill: 92 | Salary: €120M
  - FWD: Landon Powell | Skill: 89 | Salary: €110M
  - FWD: Colton Gray | Skill: 91 | Salary: €125M
Avg Skill: 87.86 | Total Salary: €735M

🏆 Team 2
  - GK: Alex Carter | Skill: 85 | Salary: €90M
  - DEF: Caleb Fisher | Skill: 84 | Salary: €85M
  - DEF: Mason Reed | Skill: 82 | Salary: €75M
  - MID: Austin Torres | Skill: 82 | Salary: €80M
  - MID: Dominic Bell | Skill: 86 | Salary: €95M
  - FWD: Xavier Bryant | Skill: 90 | Salary: €120M
  - FWD: Julian Scott | Skill: 92 | Salary: €130M
Avg Skill: 85.86 | Total Salary: €675M

🏆 Team 3
  - GK: Blake Henderson | Skill: 87 | Salary: €95M
  - DEF: Ethan Howard | Skill: 80 | Salary: €70M
  - DEF: Daniel Foster | Skill: 90 | Salary: €110M
  - MID: Bentley Rivera

In [10]:


# === PRINT POPULATION DETAILS ===
for idx, indiv in enumerate(population):
    print("\n" + "=" * 35)
    print(f"🏟️  League (Individual) {idx + 1}")
    print("=" * 35)

    for tidx, team in enumerate(indiv.league):
        print(f"\n  🏆 Team {tidx + 1}")
        print(team)
        print(f"    📊 Avg Skill: {team.avg_skill():.2f}")
        print(f"    💰 Total Salary: €{team.total_salary()}M")

    print(f"\n  ➤ League Fitness (Std Dev of team avg skill): {indiv.fitness:.4f}")



🏟️  League (Individual) 1

  🏆 Team 1
  - GK: Blake Henderson | Skill: 87 | Salary: €95M
  - DEF: Maxwell Flores | Skill: 81 | Salary: €72M
  - DEF: Jaxon Griffin | Skill: 79 | Salary: €65M
  - MID: Dominic Bell | Skill: 86 | Salary: €95M
  - MID: Austin Torres | Skill: 82 | Salary: €80M
  - FWD: Landon Powell | Skill: 89 | Salary: €110M
  - FWD: Elijah Sanders | Skill: 93 | Salary: €140M
    📊 Avg Skill: 85.29
    💰 Total Salary: €657M

  🏆 Team 2
  - GK: Alex Carter | Skill: 85 | Salary: €90M
  - DEF: Brayden Hughes | Skill: 87 | Salary: €100M
  - DEF: Logan Brooks | Skill: 86 | Salary: €95M
  - MID: Connor Hayes | Skill: 89 | Salary: €105M
  - MID: Gavin Richardson | Skill: 87 | Salary: €95M
  - FWD: Chase Murphy | Skill: 86 | Salary: €95M
  - FWD: Sebastian Perry | Skill: 95 | Salary: €150M
    📊 Avg Skill: 87.86
    💰 Total Salary: €730M

  🏆 Team 3
  - GK: Ryan Mitchell | Skill: 83 | Salary: €85M
  - DEF: Mason Reed | Skill: 82 | Salary: €75M
  - DEF: Lucas Bennett | Skill: 85 |